In [17]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random
import scipy.stats as stats

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import pandas as pd
import copy


In [2]:
import sys
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
import BeadDataFile
from discharge_tools import *

from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

sys.path.append('/home/analysis_user/New_trap_code/SensitivityFramework/')
from signal_model_utilities_v3b import *

In [3]:
%matplotlib widget

In [4]:
%reload_ext autoreload
%autoreload 2

### Parameters

In [5]:
df = pd.read_pickle("/data/new_trap_processed/processed_files/20200320/Bead1/Shaking/Shaking378/Shaking3_aux.pkl") #here one could put the respective file name
height_cal = df["z_distance_emp"]
time = df['Time_Epoch']
time -= time[0]

In [6]:
fname = r'/home/data_transfer/data_from_windows/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_X_m300k_250s_1hz.h5'
tf_x = BeadDataFile.BeadDataFile(fname)
fname = r'/home/data_transfer/data_from_windows/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Y_m300k_250s_1hz.h5'
tf_y = BeadDataFile.BeadDataFile(fname)
fname = r'/home/data_transfer/data_from_windows/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
tf_z = BeadDataFile.BeadDataFile(fname)
gfw.build_transfer_function([tf_x, tf_y, tf_z], base_freq=1, number_of_harmonics=90, scale_freq=71, plot=False)

No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data


In [7]:
bdf_discharge = load_dir(dirname='/data/new_trap/20200320/Bead1/Discharge/Discharge_after_Mass_20200402/After_spinning_again_20200403/',
                         start_file=230, max_file=40)
bdf_z = bdf_discharge[5:35]

530  files in folder
40  files loaded


In [8]:
bandwidth = 2
m1_zscale = gfw.build_z_response(bdf_z, drive_freq=71, charges=6, bandwidth=bandwidth, decimate=10)

***************************************************
Z2-amplitude:  2.69e+02
reduced chi2:  441.1817586533457
***************************************************
Z2-amplitude:  2.61e+02
reduced chi2:  489.117997931911
***************************************************
Z2-amplitude:  2.74e+02
reduced chi2:  367.35685535243334
***************************************************
Z2-amplitude:  2.79e+02
reduced chi2:  442.30175900202806
***************************************************
Z2-amplitude:  2.80e+02
reduced chi2:  462.42977088215497
***************************************************
Z2-amplitude:  2.75e+02
reduced chi2:  302.2298893684773
***************************************************
Z2-amplitude:  2.69e+02
reduced chi2:  554.2181314149083
***************************************************
Z2-amplitude:  2.65e+02
reduced chi2:  657.0098909281799
***************************************************
Z2-amplitude:  2.66e+02
reduced chi2:  512.0367862193627
*****************

In [9]:
def get_Yoffset(i):
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)    
    return (437.3 - 25 * 9.5) - (aux.voltage_to_position(np.mean(bb.cant_pos[1])))
def get_Stroke(i):
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)    
    return -np.std(aux.voltage_to_position(bb.cant_pos[1]))*np.sqrt(2)*2
def get_Dist(i):
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)    
    return 392.2 - aux.voltage_to_position(np.mean(bb.cant_pos[0]))

In [10]:
z_sep = np.mean(height_cal)
y_offset = np.mean(np.array(Parallel(n_jobs=10)(delayed(get_Yoffset)(i) for i in tqdm(range(0, 10000, 100)))))
stroke = np.mean(np.array(Parallel(n_jobs=10)(delayed(get_Stroke)(i) for i in tqdm(range(0, 10000, 100)))))
dist = np.mean(np.array(Parallel(n_jobs=10)(delayed(get_Dist)(i) for i in tqdm(range(0, 10000, 100)))))
G_lambda = 10
print(z_sep, y_offset, stroke, dist, G_lambda)

100%|██████████| 100/100 [00:00<00:00, 982.09it/s]


-15.859100393 4.88214369679 -202.765748131 13.9887662292 10


In [39]:
test_shaking378 = force_vs_time(dist,z_sep,stroke,3,G_lambda,"z", 
                                yuk_or_grav="yuk",offset_y=y_offset, alpha=1e8, bead_size=7.6)
shaking378_template = np.array(list(test_shaking378[1])*10)

Loading Gravity Data... Done!
81 9.74070978211e-06
Loaded Yukawa Force


In [41]:
def fft_norm(N, fsamp):
    "Factor to normalize FFT to ASD units"
    return np.sqrt(2 / (N * fsamp))
scale = fft_norm(5000, 50000)

_,ax = plt.subplots()
x = shaking378_template
fft = np.abs(np.fft.rfft(x))*scale
angles = np.angle(np.fft.rfft(x))
freq = np.fft.rfftfreq(len(x), d=1./5000)
ax.semilogy(freq, fft, label='7.6 $\mu m$ sphere - Z force')
ax.set(xlim=(1,100), ylim=(None,None), xlabel='frequency [Hz]', ylabel='Force [N/sqrt(Hz.)]', title=r'$\alpha=1x10^{8}$')
ax.legend()
print('Z force [N] at 36Hz.:', fft[freq==36])
print('sensitivity [alpha]:', 1e-17/fft[freq==36]*1e8)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Z force [N] at 36Hz.: [  6.23099642e-18]
sensitivity [alpha]: [  1.60487975e+08]


In [44]:
gravity_freqs = 3*np.arange(1,20)
scales = np.array([gfw.scale_Z2])* np.interp(gravity_freqs, gfw.tf_freq, gfw.tf_ffts[2])

In [45]:
fft_bits = copy.deepcopy(fft)
for freq_,scale_ in zip(gravity_freqs,scales):
    fft_bits[freq==freq_] *= scale_

In [54]:
del data_bdfs2

In [55]:
# noise estimation
data_bdfs2 = load_dir(dirname='/data/new_trap/20200320/Bead1/Shaking/Shaking2_20200403/',
                         file_prefix='NoShaking_1', start_file=0, max_file=100)
noise_x = copy.deepcopy(np.array(data_bdfs2[0].z2))
for bb in data_bdfs2[1:]:
    noise_x += copy.deepcopy(np.array(bb.z2))
noise_x = noise_x/100

6905  files in folder
100  files loaded


In [79]:
fft_noise1 = np.abs(np.fft.rfft(data_bdfs2[50].z2))*scale
fft_noise2 = np.abs(np.fft.rfft(noise_x))*scale

In [78]:
_,ax = plt.subplots()
# ax.semilogy(freq, fft_bits, label=r'$\alpha$=$10^8$')
ax.semilogy(freq, fft_noise1/np.sqrt(5), label='1 file - noise')
# ax.semilogy(freq, fft_noise2/10, label='100 files - noise')
ax.set(xlim=(1,100), ylim=(None,None), xlabel='frequency [Hz]', ylabel='Force [bits]', title=r'$\alpha=1x10^{8}$')
ax.legend()
print('Z force [N] at 36Hz.:', fft[freq==36])
print('sensitivity [alpha]:', 1e-17/fft[freq==36]*1e8)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Z force [N] at 36Hz.: [  6.23099642e-18]
sensitivity [alpha]: [  1.60487975e+08]


In [70]:
2**16

65536

In [71]:
_,ax = plt.subplots()
ax.plot(range(1000), data_bdfs2[10].z2[:1000]-np.mean(data_bdfs2[10].z2))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …